In [ ]:
import warnings

warnings.filterwarnings(action='ignore', category=DeprecationWarning)

import sys
import numpy as np

import mbuild
import foyer

import parmed
import parmed.unit as u

import openmm.app
import openmm 

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
class MSDReporter:
    def __init__(self, interval, simulation, dt):
        self.interval = interval
        state0 = simulation.context.getState(getPositions=True,
                                             enforcePeriodicBox=False)
        self.x0 = state0.getPositions(asNumpy=True).value_in_unit(u.centimeter)
        self.dt = dt.value_in_unit(u.second)
        self.label = dict(Time=[], MSD=[])

    def describeNextReport(self, simulation):
        steps = self.interval - simulation.currentStep % self.interval
        return (steps, True, False, False, False, False)

    def report(self, simulation, state):
        x = state.getPositions(asNumpy=True).value_in_unit(u.centimeter)
        msd = np.mean(np.linalg.norm((x - self.x0), axis=1)**2)
        time = state.getStepCount() * self.dt
        self.label['Time'].append(time)
        self.label['MSD'].append(msd)

In [ ]:
# butanol1 = parmed.load_file(r'Butanol-1\Structure.pdb')
# _ff = foyer.Forcefield(r'Butanol-1\FF.xml')
# # _ff = foyer.forcefields.load_OPLSAA()
# butanol1 = _ff.apply(butanol1)

In [ ]:
oplsaa = foyer.forcefields.load_OPLSAA()

# butanol1 = mbuild.load(r'Butanol-1\Structure_simple.pdb')
butanol1 = mbuild.load('CCCCO',smiles=True)
butanol1.energy_minimize()
butanol1.name = 'Butanol'

ocm2 = mbuild.load(r'OCM-2\Structure_simple.pdb')
ocm2 = mbuild.load('CC(C)C(OCCOC(OCCOCCOC(OCCOC(C(C)C)=O)=O)=O)=O',smiles=True)
ocm2.energy_minimize()
ocm2.name = 'OCM'

In [ ]:
## Combined box structure
print('Build butanol-1 box')
box_but = mbuild.Box([5, 5, 5])

box_but2 = mbuild.fill_box(
    box=box_but,
    compound=butanol1,
    n_compounds=100,
)
print('Build ocm-2 box')
box_ocm = mbuild.Box([5, 5, 5])
box_ocm2 = mbuild.fill_box(
    box=box_ocm,
    compound=ocm2,
    n_compounds=100,
)

print('Appling on butanol')
box_but3 = oplsaa.apply(box_but2)

print('Appling on ocm')
box_ocm3 = oplsaa.apply(box_ocm2)

print('Combining')
structure = box_ocm3 + box_but3

In [ ]:
## Filled box structure
# box = mbuild.fill_box(
#     compound=[butanol1,ocm2],
#     compound_ratio=[0.5,0.5],
#     box=[5,5,5],
#     density=1000
# )

box = mbuild.Box([5, 5, 5])
box2 = mbuild.fill_box(
    compound=ocm2,
    n_compounds=100,
    box=box,
)
box3 = mbuild.solvate(solute=box2, solvent=butanol1, n_solvent=100, box=box)
structure = oplsaa.apply(box3)

In [ ]:
structure.save('initial.pdb')

In [ ]:
structure.residues[0].name='A'
structure.residues[1].name='B'

In [ ]:
system = structure.createSystem(
    nonbondedMethod=openmm.app.NoCutoff,
    constraints=openmm.app.HBonds,
    implicitSolvent=openmm.app.GBn2,
    implicitSolventSaltConc=0.1 * u.moles / u.liter,
    removeCMMotion=False,
)


In [ ]:
integrator = openmm.LangevinIntegrator(
    300 * u.kelvin,  # Temperature of heat bath
    1.0 / u.picoseconds,  # Friction coefficient
    2.0 * u.femtoseconds,  # Time step
)
simulation = openmm.app.Simulation(
    structure.topology,
    system,
    integrator,
    openmm.Platform.getPlatformByName('CUDA'),
)
simulation.context.setPositions(structure.positions)
simulation.minimizeEnergy(maxIterations=500)

In [ ]:
simulation.reporters.append(
    openmm.app.StateDataReporter(
        sys.stdout,
        1000,
        step=True,
        # potentialEnergy=True,
        # kineticEnergy=True,
        # temperature=True,
    ))

rep_D = openmm.app.DCDReporter(
    'traj.dcd',
    100,
    # enforcePeriodicBox=False,
)
simulation.reporters.append(rep_D)

rep_pdb = openmm.app.PDBReporter('system.pdb',100)
simulation.reporters.append(rep_pdb)

# rep_old = MSDReporter(100, simulation, 2.0 * u.femtosecond)
# simulation.reporters.append(rep_old)

In [ ]:
simulation.step(100000)

In [ ]:
df = pd.DataFrame(rep.label)


In [ ]:
plt.plot(df['Time'], df['MSD'])

# trajectory

In [ ]:
import mdtraj

In [ ]:
trajectory = mdtraj.load_pdb('system.pdb')

In [ ]:
trajectory.superpose()

In [ ]:
a= mdtraj.rmsd(trajectory,trajectory,0)

In [ ]:
trajectory

# trajectory2

In [ ]:
import MDAnalysis
import MDAnalysis.analysis.rms

In [ ]:
trajectory = MDAnalysis.Universe('system.pdb')

In [ ]:
list(trajectory.residues)

In [ ]:
trajectory.

In [ ]:
RMSD_analysis = MDAnalysis.analysis.rms.RMSD(
    trajectory,
    trajectory,
    # select="backbone",
    groupselections=["resid 1", "resid 2"])

In [ ]:
RMSD_analysis.run()

In [ ]:
result = RMSD_analysis.results.rmsd.T

In [ ]:
step = result[0]
time = result[1]
msd_all = result[2]
msd_A = result[3]
msd_B = result[4]

In [ ]:
plt.plot(time, msd_all, 'b--', label="all")
plt.plot(time, msd_A, 'r', label="A")
plt.plot(time, msd_B, 'g', label="B")
plt.legend()

In [ ]:
from scipy.stats import linregress

In [ ]:
regress = linregress(time,msd_A)
D_A = regress.slope/6

regress = linregress(time,msd_B)
D_B = regress.slope/6

regress = linregress(time,msd_all)
D_all = regress.slope/6
print(D_A,D_B,D_all)